# Risk Parity Approach to Equity Asset Allocation

In [ ]:
import pandas_datareader.data as web
import datetime
from scipy.optimize import minimize
TOLERANCE = 1e-10

In [ ]:
## Data Input
# Initial weights: equally weighted
init_weights = [1 / X_train.shape[1]] * X_train.shape[1]
covariances = returns.cov()

# Key difference to inverse volatility models:
# The desired contribution of each asset to the portfolio risk: we want all
# asset to contribute equally (verifying this part)
assets_risk_budget = [1 / X_train.shape[1]] * X_train.shape[1]

# convert dataframe to arrays
init_weights = np.array(init_weights)
covariances = covariances.values

In [ ]:
np.multiply(init_weights.T, covariances * init_weights.T)
init_weights*covariances * init_weights.T

In [ ]:
def _allocation_risk(weights, covariances):

    # portfolio_risk = np.sqrt((weights * covariances * weights.T))
    portfolio_risk = np.sqrt((weights * covariances * weights.T))[0, 0]

    return portfolio_risk

def _assets_risk_contribution_to_allocation_risk(weights, covariances):

    # risk of the weights distribution
    portfolio_risk = _allocation_risk(weights, covariances)

    # the contribution of each asset to the risk
    assets_risk_contribution = np.multiply(weights.T, covariances * weights.T) \
        / portfolio_risk

    return assets_risk_contribution

def _risk_budget_objective_error(weights, args):

    covariances = args[0]
    assets_risk_budget = args[1]
    weights = np.matrix(weights)

    portfolio_risk = _allocation_risk(weights, covariances)

    # the contribution of each asset to the risk of the weights distribution
    assets_risk_contribution = \
        _assets_risk_contribution_to_allocation_risk(weights, covariances)

    # the desired contribution of each asset to the risk of the
    # weights distribution
    assets_risk_target = \
        np.asmatrix(np.multiply(portfolio_risk, assets_risk_budget))

    # Error between the desired contribution and the calculated contribution of
    # each asset (to be minimized)
    error = \
        sum(np.square(assets_risk_contribution - assets_risk_target.T))[0, 0]

    return error

def _get_risk_parity_weights(covariances, assets_risk_budget, initial_weights):

    # Restrictions to consider in the optimisation: only long positions whose
    # sum equals 100%
    constraints = ({'type': 'eq', 'fun': lambda x: np.sum(x) - 1.0},
                   {'type': 'ineq', 'fun': lambda x: x})

    # Optimisation process in scipy
    optimize_result = minimize(fun=_risk_budget_objective_error,
                               x0=initial_weights,
                               args=[covariances, assets_risk_budget],
                               method='SLSQP',
                               constraints=constraints,
                               tol=TOLERANCE,
                               options={'disp': False})

    # Recover the weights from the optimised object
    weights = optimize_result.x

    return weights

def get_weights(covariances, assets_risk_budget, init_weights):

    # Optimisation process of weights
    weights = \
        _get_risk_parity_weights(covariances, assets_risk_budget, init_weights)

    # Convert the weights to a pandas Series
    weights = pd.Series(weights, index=X_train.columns, name='weight')

    return weights

In [ ]:
get_weights(covariances, assets_risk_budget, init_weights)